In [1]:
# ===============================
# Interactive Visual Analytics with Folium - Full Setup
# ===============================

# Install required libraries (uncomment if not installed)
# !pip install folium pandas

# Import libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# -------------------------------
# Fetch the SpaceX launch sites data from URL
# -------------------------------
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
spacex_df = pd.read_csv(url)

# Select relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Get unique launch sites
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Display launch sites
launch_sites_df

# -------------------------------
# Create a folium map centered at NASA Johnson Space Center
# -------------------------------
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add each launch site as a Circle and Marker
for idx, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    
    # Add Circle
    circle = folium.Circle(
        location=coord,
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Add Marker with label
    marker = folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


In [2]:
# Task 2: Mark the success/failed launches for each site

from folium.plugins import MarkerCluster

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Create a new column 'marker_color' based on launch outcome
# green for success (class=1), red for failure (class=0)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Add each launch as a marker to the cluster
for idx, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket'),
        popup=(
            f"Launch Site: {record['Launch Site']}<br>"
            f"Outcome: {'Success' if record['class']==1 else 'Failure'}"
        )
    )
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map


In [4]:
# TASK 3a: Calculate the distances between a launch site to its proximities

# Install geopy if not already installed
!pip install geopy

from geopy.distance import great_circle

# Example: Define a point of interest, e.g., the closest coastline, a city, or another site
# Here, we'll calculate distance from each launch site to the NASA Johnson Space Center (nasa_coordinate)
launch_sites_df['Distance_to_NASA_JSC_km'] = launch_sites_df.apply(
    lambda row: great_circle(
        (row['Lat'], row['Long']), 
        nasa_coordinate
    ).kilometers, axis=1
)

# Display the distances
launch_sites_df[['Launch Site', 'Distance_to_NASA_JSC_km']]



   ---------------------------------------- 0/2 [geographiclib]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   ---------------------------------------- 2/2 [geopy]



,Launch Site,Distance_to_NASA_JSC_km
0,CCAFS LC-40,1413.330588
1,CCAFS SLC-40,1413.368598
2,KSC LC-39A,1406.436257
3,VAFB SLC-4E,2462.757200


In [6]:
from math import sin, cos, sqrt, atan2, radians

# Function to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Example closest coastline coordinates (replace with your actual points from MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Offset for markers so they don't overlap
offset = 0.02

for i, site in enumerate(launch_sites_df.itertuples()):
    site_coord = [site.Lat, site.Long]
    distance_km = calculate_distance(site.Lat, site.Long, coastline_lat, coastline_lon)
    
    # Offset the marker for readability
    marker_coord = [coastline_lat + i*offset, coastline_lon + i*offset]
    
    # Add distance marker
    distance_marker = folium.Marker(
        location=marker_coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
        )
    )
    site_map.add_child(distance_marker)
    
    # Draw line from launch site to coastline
    line = folium.PolyLine(
        locations=[site_coord, [coastline_lat, coastline_lon]],
        weight=2,
        color='blue'
    )
    site_map.add_child(line)

site_map


In [7]:
# Example coordinates for nearby points (replace with actual coordinates from MousePosition)
proximities = {
    'Coastline': (28.56367, -80.57163),
    'Railway': (28.56500, -80.57500),
    'Highway': (28.56050, -80.57000),
    'City': (28.57000, -80.58000)
}

# Offset increment for markers so they don't overlap
offset = 0.02

for i, site in enumerate(launch_sites_df.itertuples()):
    site_coord = [site.Lat, site.Long]
    
    for j, (label, coords) in enumerate(proximities.items()):
        prox_lat, prox_lon = coords
        distance_km = calculate_distance(site.Lat, site.Long, prox_lat, prox_lon)
        
        # Offset markers for readability
        marker_coord = [prox_lat + (i+j)*offset, prox_lon + (i+j)*offset]
        
        # Add distance marker
        distance_marker = folium.Marker(
            location=marker_coord,
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html='<div style="font-size: 12; color:#d35400;"><b>{}: {:.2f} KM</b></div>'.format(label, distance_km)
            )
        )
        site_map.add_child(distance_marker)
        
        # Draw line from launch site to proximity
        line = folium.PolyLine(
            locations=[site_coord, [prox_lat, prox_lon]],
            weight=2,
            color='blue'
        )
        site_map.add_child(line)

site_map
